In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# 
# Programa do curso Técnicas de raspagem de dados e acesso a APIs
#
# Alguns exemplos de como usar a API do Twitter para acessar tweets da rede social


In [4]:
import tweepy
import pandas as pd
import csv

In [2]:
# Digita chaves de acesso pessoais do seu Twitter - obtidas em https://developer.twitter.com

In [3]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

In [4]:
# Autenticações na API do Twitter

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [6]:
# Acessa os tweets recentes de um usuário

In [7]:
lista = api.user_timeline('jairbolsonaro')

In [8]:
# Mostra na tela

In [9]:
for status in lista:
    print (status._json)

{'created_at': 'Wed May 29 20:59:10 +0000 2019', 'id': 1133840226672095234, 'id_str': '1133840226672095234', 'text': '- Recebi hoje cedo uma visita mais que especial dos alunos da escola Céu Azul, de Valparaíso de Goiás. A essas cria… https://t.co/Kfxg6Gg7WY', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/Kfxg6Gg7WY', 'expanded_url': 'https://twitter.com/i/web/status/1133840226672095234', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 128372940, 'id_str': '128372940', 'name': 'Jair M. Bolsonaro', 'screen_name': 'jairbolsonaro', 'location': 'Brasília, Brasil', 'description': 'Capitão do Exército Brasileiro, eleito 38° Preside

In [ ]:
# Estratégia para armazenar os Tweets mais recentes
# Programa pode ser rodado todos os dias

In [24]:
def get_all_tweets(screen_name, partido, politico, cargo):
    
    #Inicializa uma lista para conter todos os tweets possíveis
    alltweets = []  

    #Faça o pedido inicial para os tweets mais recentes (200 é a contagem máxima permitida)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)

    #salvar os tweets mais recentes
    alltweets.extend(new_tweets)

    #salvar o id do tweet mais antigo menos um
    oldest = alltweets[-1].id - 1

    # Continue segurando os tweets até que não haja mais tweets para pegar
    while len(new_tweets) > 0:
        #print "getting tweets before %s" % (oldest)

        # Todos os pedidos subsequentes usam max_id para evitar duplicatas
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        # Salva os mais recentes tweets
        alltweets.extend(new_tweets)

        # atualizar o id do tweet mais antigo menos um
        oldest = alltweets[-1].id - 1

        #print "...% s tweets baixados até agora"% (len (alltweets))

    # Transforme os tweets tweepy em uma matriz 2D que preencherá o csv
    # outra opção: tweet.text.encode ("utf-8")
    # Podem ser adicionados mais campos da API do Twitter no arquivo final, não apenas esses 3 
    outtweets = [[screen_name, partido, politico, tweet.id_str, tweet.created_at, tweet.text, cargo] for tweet in alltweets]

    #Escreve o CSV
    with open('resultados/%s_tweets.csv' % screen_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(["conta_twitter", "partido", "politico", "id","created_at","text", "cargo"])
        writer.writerows(outtweets)

    pass

In [25]:
if __name__ == '__main__':
    # Neste exemplo o arquivo é lido para pegar o endereço de Twitter, pode ser um arquivo com qualquer endereço
    candidatos = pd.read_excel('redes_sociais_politicos_exemplo.xlsx',sheet_name='Planilha1')
    candidatos.reset_index().head()
    
    for num, row in candidatos.iterrows():
        partido = (row['Partido']).strip()
        politico = (row['Nome']).strip()
        
        print(politico)
        
        endereco = (row['Twitter']).strip()
        cargo = (row['Cargo (ultima alteração)']).strip()
        get_all_tweets(endereco, partido, politico, cargo)

Jader Fontenelle Barbalho
Sergio Olimpio Gomes
Airton Luiz Faleiro
Jair Messias Bolsonaro
